<a href="https://colab.research.google.com/github/rishalas/python-Julia-from-Power-System-GAMS-book/blob/main/Gcode32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Example of sensitivity analysis regarding load value in ED

In [1]:
!wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
!unzip -o -q ipopt-linux64
!pip install pyomo
from pyomo.environ import *
import matplotlib.pyplot as plt
import numpy as np
import random 

     |████████████████████████████████| 9.1 MB 14.9 MB/s 
     |████████████████████████████████| 49 kB 3.6 MB/s 


In [22]:
# making variable

model = AbstractModel()
model.gen=Set()
model.a=Param(model.gen)
model.b=Param(model.gen)
model.c=Param(model.gen)
model.Pmin=Param(model.gen)
model.Pmax=Param(model.gen)
model.l =Param(initialize=400,mutable=True)


# making Objective and Constraint
def Pbounds(model,gen):
  return (model.Pmin[gen],model.Pmax[gen]);

model.p=Var(model.gen,bounds=Pbounds,within=Reals);
model.of=Var(within=Reals);

def rule1(model,gen):
  return sum(model.p[gen] for gen in model.gen) >= model.l;
model.c1 = Constraint(model.gen,rule=rule1);

def ruleof(model,gen):
  return model.of ==sum((model.a[gen]*model.p[gen]*model.p[gen] + 
                             model.b[gen]*model.p[gen]+model.c[gen]) for gen in model.gen)
model.c3= Constraint(rule=ruleof)
model.obj=Objective(expr=model.of,sense=minimize)

# input file
from google.colab import files
  
  
uploaded = files.upload()

#input data and solver , credit for Dr Sanjula Kammamettu for helping solve this counter function

#print('  OF', ' load')
Nsteps = 11;
for counter in range(1,Nsteps+1):
    instance= model.create_instance("gcode31.dat")
    instance.l = sum (instance.Pmin[gen] for gen in instance.gen) +(counter-1)/(Nsteps-1)*sum(instance.Pmax[gen]-instance.Pmin[gen] for gen in instance.gen);
    print('Load= ',instance.l());
    results = SolverFactory('ipopt', executable='/content/ipopt').solve(instance)
    print('OF =   ',value(instance.of()));
    #instance.pprint()
    #results.write()
    #print("%5.2f"%value(instance.l),"%5.2f"%value(instance.of))
    instance.p.pprint()

Saving gcode31.dat to gcode31 (18).dat
Load=  281.0
OF =    84037.84838139187
p : Size=5, Index=gen
    Key : Lower : Value : Upper : Fixed : Stale : Domain
     G1 :    28 :  28.0 :   206 : False : False :  Reals
     G2 :    90 :  90.0 :   284 : False : False :  Reals
     G3 :    68 :  68.0 :   189 : False : False :  Reals
     G4 :    76 :  76.0 :   266 : False : False :  Reals
     G5 :    19 :  19.0 :    53 : False : False :  Reals
Load=  352.7
OF =    105758.01853018063
p : Size=5, Index=gen
    Key : Lower : Value              : Upper : Fixed : Stale : Domain
     G1 :    28 :  67.39040630743212 :   206 : False : False :  Reals
     G2 :    90 :               90.0 :   284 : False : False :  Reals
     G3 :    68 :               68.0 :   189 : False : False :  Reals
     G4 :    76 :               76.0 :   266 : False : False :  Reals
     G5 :    19 : 51.309592505559564 :    53 : False : False :  Reals
Load=  424.4
OF =    147715.93892781308
p : Size=5, Index=gen
    Key : Lowe